In [1]:
"""
图像卷积
"""

'\n图像卷积\n'

In [3]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):
    '''计算二维互相关运算'''
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [4]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [6]:
#基于上面定义的corr2d函数实现二维卷积层
class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        super.__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self,x):
        return corr2d(x,self.weight) + self.bias

In [7]:
# 图像中目标的边缘检测
# 如下是卷积层的一个简单应用：通过找到像素变化的位置，来检测图像中不同颜色的边缘，首先，我们构造一个\(6\times 8\)像素的黑白图像。中间四列为黑色(0），其余像素为白色（1）
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [8]:
k = torch.tensor([[1.0,-1.0]])

In [10]:
Y = corr2d(X,k)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [12]:
#检测转置的
corr2d(X.t(),k)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [13]:
#学习由X生成Y的卷积核
conv2d = nn.Conv2d(1,1,kernel_size=(1,2),bias=False)
# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = X.reshape((1,1,6,8))
Y = Y.reshape((1,1,6,7))
lr = 3e-2

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat-Y)**2
    conv2d.zero_grad()#将之前计算的梯度清零
    l.sum().backward()#对损失函数进行反向传播
    conv2d.weight.data[:] -= lr * conv2d.weight.grad#更新权重
    if (i+1)%2==0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 9.391
epoch 4, loss 2.567
epoch 6, loss 0.837
epoch 8, loss 0.307
epoch 10, loss 0.120


In [15]:
#查看学习到的卷积核
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9529, -1.0231]])